In [51]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import requests
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# BeautifulSoup
Saya mencoba menggunakan beautiful soup untuk melihat apakah website ini statis atau dinamis. Sepertinya web ini dinamis karena isi konten nya tidak langsung keluar menggunakan html parser biasa.

In [2]:
url = "https://ocw.mit.edu/search/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
# print(soup)
# Jika kita print akan sulit melihat isi konten courses

# Selenium

Selenium berfungsi lebih baik untuk konten dinamis. Namun karena website nya dynamically loaded, kita tidak bisa mendapatkan semua konten course (~2000). Saya mencoba melakukan scroll down terlebih dahulu untuk mendapatkan semua konten, namun tetap tidak bisa mendapatkan semua course, hanya ratusan saja.

In [67]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [68]:
def scroll_down(driver):
    """A method for scrolling the page."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to the bottom.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load the page.
        time.sleep(6)
        # Calculate new scroll height and compare with last scroll height.
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [103]:
driver.get("https://ocw.mit.edu/search/")

WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@aria-label="OpenCourseWare Search Results"]'))
)

scroll_down(driver)

wait = WebDriverWait(driver, 10)

url_elements = driver.find_elements(By.XPATH, '//*[@class="lr-row course-title"]/a')
course_urls = []

for el in url_elements:
    course_url = el.get_attribute('href')
    course_urls.append(course_url)

In [ ]:
len(course_urls)

710

Fetched 710 URLs/courses. Next, loop through all URLs to get title, description, etc.

In [ ]:
courses = []
for url in course_urls:
    driver.get(url)
    
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'course-number-term-detail'))
    )
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    number_term_level = soup.select_one('.course-number-term-detail').text
    number = number_term_level.split(' | ')[0]
    semester = number_term_level.split(' | ')[1].split()[0]
    year = number_term_level.split(' | ')[1].split()[1]
    level = number_term_level.split(' | ')[2]
    title = soup.select_one('#course-banner > div > div > div:nth-of-type(1) > h1 > a').text.strip()
    description = soup.select('.description')[-1].text.replace("Show less", "")
    instructors = [el.text for el in soup.select('#course-main-content .course-info-instructor')]
    departments = [el.text for el in soup.select('#course-main-content .course-info-department')]
    topics = [el.text for el in soup.select('#course-main-content .course-info-topic')]
   
    courses.append({"number": number,
                    "semester": semester,
                    "year":year,
                    "level": level,
                    "title": title,
                    "description": description,
                    "instructors": instructors,
                    "departments": departments,
                    "topics": topics})
    
    time.sleep(4)

In [95]:
courses

[{'number': '14.44',
  'term': 'Spring 2007',
  'level': 'Undergraduate',
  'title': 'Energy Economics',
  'description': 'This course explores the theoretical and empirical perspectives on individual and industrial demand for energy, energy supply, energy markets, and public policies affecting energy markets. It discusses aspects of the oil, natural gas, electricity, and nuclear power sectors and examines energy tax, price regulation, deregulation, energy efficiency and policies for controlling emission.',
  'instructors': ['Prof. Paul Joskow'],
  'departments': ['Economics'],
  'topics': ['Energy',
   'Electricity',
   'Fossil Fuels',
   'Nuclear',
   'Science',
   'Earth Science',
   'Sustainability',
   'Social Science',
   'Economics',
   'Microeconomics',
   'Political Economy',
   'Public Administration',
   'Environmental Policy']},
 {'number': '21A.101J (formerly 21A.218J)',
  'term': 'Spring 2010',
  'level': 'Undergraduate',
  'title': 'Identity and Difference',
  'descripti

# API
API ditemukan pada network tab saat inspect element.

In [96]:
url = "https://open.mit.edu/api/v0/search/"
# change the size parameters to course max count
body = {"from":0,"size": 2567,"post_filter":{"bool":{"must":[{"bool":{"should":[{"term":{"object_type.keyword":"course"}}]}},{"bool":{"should":[{"term":{"offered_by":"OCW"}}]}}]}},"query":{"bool":{"should":[{"bool":{"filter":{"bool":{"must":[{"term":{"object_type":"course"}}]}}}}]}},"aggs":{"agg_filter_topics":{"filter":{"bool":{"should":[{"bool":{"filter":{"bool":{"must":[{"bool":{"should":[{"term":{"object_type.keyword":"course"}}]}},{"bool":{"should":[{"term":{"offered_by":"OCW"}}]}}]}}}}]}},"aggs":{"topics":{"terms":{"field":"topics","size":10000}}}},"agg_filter_department_name":{"filter":{"bool":{"should":[{"bool":{"filter":{"bool":{"must":[{"bool":{"should":[{"term":{"object_type.keyword":"course"}}]}},{"bool":{"should":[{"term":{"offered_by":"OCW"}}]}}]}}}}]}},"aggs":{"department_name":{"terms":{"field":"department_name","size":10000}}}},"agg_filter_level":{"filter":{"bool":{"should":[{"bool":{"filter":{"bool":{"must":[{"bool":{"should":[{"term":{"object_type.keyword":"course"}}]}},{"bool":{"should":[{"term":{"offered_by":"OCW"}}]}}]}}}}]}},"aggs":{"level":{"nested":{"path":"runs"},"aggs":{"level":{"terms":{"field":"runs.level","size":10000},"aggs":{"courses":{"reverse_nested":{}}}}}}}},"agg_filter_course_feature_tags":{"filter":{"bool":{"should":[{"bool":{"filter":{"bool":{"must":[{"bool":{"should":[{"term":{"object_type.keyword":"course"}}]}},{"bool":{"should":[{"term":{"offered_by":"OCW"}}]}}]}}}}]}},"aggs":{"course_feature_tags":{"terms":{"field":"course_feature_tags","size":10000}}}}}}
headers = {
    # introduce as bot
    "User-Agent": "Scraping Bot", 
    "Accept": "application/json"
}
result = requests.post(url, json = body)
result = result.json()


In [99]:
api_courses = []

for data in result['hits']['hits']:
    data = data['_source']
    course = {}
    course['number'] = data['coursenum'].split()[0]
    course['title'] = data['title']
    course['level'] = data['runs'][0]['level']
    course['semester'] = data['runs'][0]['semester']
    course['year'] = data['runs'][0]['year']
    course['description'] = data['runs'][0]['short_description']
    course['topics'] = data['topics']
    course['instructors'] = data['runs'][0]['instructors']
    course['department_name'] = data['department_name']

    api_courses.append(course)

In [100]:
api_courses

[{'number': '14.44',
  'title': 'Energy Economics',
  'level': ['Undergraduate'],
  'semester': 'Spring',
  'year': 2007,
  'description': 'This course explores the theoretical and empirical perspectives on individual and industrial demand for energy, energy supply, energy markets, and public policies affecting energy markets. It discusses aspects of the oil, natural gas, electricity, and nuclear power sectors and examines energy tax, price regulation, deregulation, energy efficiency and policies for controlling emission.',
  'topics': ['Science',
   'Economics',
   'Social Science',
   'Public Administration',
   'Earth Science',
   'Energy',
   'Microeconomics',
   'Political Economy',
   'Fossil Fuels',
   'Nuclear',
   'Electricity',
   'Environmental Policy',
   'Sustainability'],
  'instructors': ['Prof. Paul Joskow'],
  'department_name': ['Economics']},
 {'number': '21A.101J',
  'title': 'Identity and Difference',
  'level': ['Undergraduate'],
  'semester': 'Spring',
  'year': 

# Text Processing

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')

def lowering(text: str) -> str:
    text = text.lower()
    return text

def remove_punctuation_and_symbol(text: str) -> str:
    text = re.sub(r'[^\w\s]', '', text)
    return text

def stopword_removal(text: str) -> str:
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

def stemming(text: str) -> str:
    text = " ".join([stemmer.stem(word) for word in text.split()])
    return text

def lemmatization(text: str) -> str:
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

def preprocessing(text: str) -> str:

    text = lowering(text)
    text = remove_punctuation_and_symbol(text)
    text = stopword_removal(text)
    text = stemming(text)
    text = lemmatization(text)

    return text

In [ ]:
# text fields that can be used for text vectorization are title, description, and topics
# for description, we will apply all preprocessing steps
# for title and topics, we will only apply lowering and remove punctuation and symbol


for course in api_courses:
    course['description'] = preprocessing(course['description'])
    course['title'] = remove_punctuation_and_symbol(lowering(course['title']))
    course['topics'] = [remove_punctuation_and_symbol(lowering(topic)) for topic in course['topics']]

# Store Data
Akan disimpan sebagai CSV karena lebih mudah untuk data analysis

In [ ]:
import pandas as pd

# convert list to string separated by ", "
for course in api_courses:
    course['instructors'] = ", ".join(course['instructors'])
    course['department_name'] = ", ".join(course['department_name'])
    course['topics'] = ", ".join(course['topics'])

courses_df = pd.DataFrame(courses)